In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [74]:
data=pd.read_csv("hackathon.csv")
data.head()

,id,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,...,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies,rent,deposit
0,ff8081815917971401591af8895032d0,BHK2,Jayanagara,2018-07-05 17:09:49,12.936601,77.576914,FAMILY,0,0,0,...,S,2.0,2,2.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORP_BORE,IF,1.0,22000,220000.0
1,ff80818157dbe1fb0157dc4aa07513dc,BHK2,Basaveshwar Nagar,2017-12-30 17:22:12,12.997989,77.545219,FAMILY,0,0,0,...,W,2.0,0,1.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORPORATION,IH,0.0,20000,200000.0
2,ff8081815d304406015d30edcd7c4f44,BHK3,"Jaya Nagar East,Jayanagar",2017-07-11 15:42:05,12.935696,77.587642,FAMILY,0,1,0,...,E,3.0,4,9.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":true,""AC"":...",CORPORATION,AP,2.0,38000,250000.0
3,ff8081815b106986015b154d908f2ac2,BHK2,Murugeshpalya,2018-07-07 14:26:14,12.953507,77.656118,FAMILY,0,1,0,...,E,2.0,4,4.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":true,""AC"":...",BOREWELL,AP,1.0,30000,300000.0
4,ff8081815e80b789015e84a7a42a62a3,BHK2,Whitefield,2017-09-27 12:19:28,12.968520,77.742436,ANYONE,1,1,1,...,E,2.0,1,9.0,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",CORP_BORE,AP,2.0,26500,150000.0


In [75]:
#dropping the id column and changing the type of column type to float
data=data.drop("id",axis=1)
import random
data['type'].replace('BHK1',1,inplace=True)
data['type'].replace('BHK2',2,inplace=True)
data['type'].replace('BHK3',3,inplace=True)
data['type'].replace('BHK4',4,inplace=True)
data['type'].replace('RK1',1,inplace=True)
data['type'].replace('BHK4PLUS',random.choice([4-9]),inplace=True)

In [76]:
#calcultaing the skewness and kurtosis
print("Skewness: %f" % data['rent'].skew())
print("Kurtosis: %f" % data['rent'].kurt())

Skewness: 1.065753
Kurtosis: 1.046220


In [77]:
#convert all null values ti integer nan
#find the total count of missing data in all the columns

data=data.replace({'Not Available':np.nan})
mis_val=data.isnull().sum()
mis_val_percent=(100*mis_val)/len(data)
mis_val_table=pd.concat([mis_val,mis_val_percent],axis=1)
mis_val_ren_columns=mis_val_table.rename(columns={0:'missing values',1:"% of total values"})
mis_val_ren_columns=mis_val_ren_columns[mis_val_ren_columns.iloc[:,1]!=0].sort_values('% of total values',ascending=False).round(1)
#print("your selected dataframe has "+str (data.shape[1])+"columns\n" "columns that have missing value.")
#print(mis_val_ren_columns)

#drop columns with more than 50% missing values
half_count = len(data) / 2
data= data.dropna(thresh=half_count,axis=1)

In [78]:
names = data.columns.tolist()
names[names.index("type")] = 'bedroomtotal'
data.columns = names

In [79]:
obj_data = data.select_dtypes(include=['object']).copy()
obj_data.head()

,locality,activation_date,lease_type,furnishing,parking,facing,amenities,water_supply,building_type
0,Jayanagara,2018-07-05 17:09:49,FAMILY,SEMI_FURNISHED,TWO_WHEELER,S,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORP_BORE,IF
1,Basaveshwar Nagar,2017-12-30 17:22:12,FAMILY,SEMI_FURNISHED,BOTH,W,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORPORATION,IH
2,"Jaya Nagar East,Jayanagar",2017-07-11 15:42:05,FAMILY,SEMI_FURNISHED,BOTH,E,"{""LIFT"":true,""GYM"":false,""INTERNET"":true,""AC"":...",CORPORATION,AP
3,Murugeshpalya,2018-07-07 14:26:14,FAMILY,SEMI_FURNISHED,BOTH,E,"{""LIFT"":true,""GYM"":false,""INTERNET"":true,""AC"":...",BOREWELL,AP
4,Whitefield,2017-09-27 12:19:28,ANYONE,SEMI_FURNISHED,BOTH,E,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",CORP_BORE,AP


In [80]:
#data=data.drop([])

In [81]:

from sklearn.preprocessing import LabelEncoder
l = LabelEncoder()
data["lease_type"] = l.fit_transform(data["lease_type"])
data["furnishing"] = l.fit_transform(data["furnishing"])
data["parking"] = l.fit_transform(data["parking"])
data["water_supply"] = l.fit_transform(data["water_supply"])
#data.head()
dummies=pd.get_dummies(data, columns=["lease_type","furnishing","parking","water_supply"]).head()

data.head()

,bedroomtotal,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,...,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies,rent,deposit
0,2,Jayanagara,2018-07-05 17:09:49,12.936601,77.576914,3,0,0,0,0,...,S,2.0,2,2.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",2,IF,1.0,22000,220000.0
1,2,Basaveshwar Nagar,2017-12-30 17:22:12,12.997989,77.545219,3,0,0,0,1,...,W,2.0,0,1.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",1,IH,0.0,20000,200000.0
2,3,"Jaya Nagar East,Jayanagar",2017-07-11 15:42:05,12.935696,77.587642,3,0,1,0,0,...,E,3.0,4,9.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":true,""AC"":...",1,AP,2.0,38000,250000.0
3,2,Murugeshpalya,2018-07-07 14:26:14,12.953507,77.656118,3,0,1,0,1,...,E,2.0,4,4.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":true,""AC"":...",0,AP,1.0,30000,300000.0
4,2,Whitefield,2017-09-27 12:19:28,12.968520,77.742436,0,1,1,1,1,...,E,2.0,1,9.0,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",2,AP,2.0,26500,150000.0


In [82]:
data=data.drop(["locality","activation_date"],axis=1)

In [83]:
#data=data.drop(["lease_type_0","furnishing_0","parking_0","water_supply_0"],axis=1)
data.head()

,bedroomtotal,latitude,longitude,lease_type,gym,lift,swimming_pool,negotiable,furnishing,parking,...,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies,rent,deposit
0,2,12.936601,77.576914,3,0,0,0,0,2,3,...,S,2.0,2,2.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",2,IF,1.0,22000,220000.0
1,2,12.997989,77.545219,3,0,0,0,1,2,0,...,W,2.0,0,1.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",1,IH,0.0,20000,200000.0
2,3,12.935696,77.587642,3,0,1,0,0,2,0,...,E,3.0,4,9.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":true,""AC"":...",1,AP,2.0,38000,250000.0
3,2,12.953507,77.656118,3,0,1,0,1,2,0,...,E,2.0,4,4.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":true,""AC"":...",0,AP,1.0,30000,300000.0
4,2,12.968520,77.742436,0,1,1,1,1,2,0,...,E,2.0,1,9.0,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",2,AP,2.0,26500,150000.0


In [84]:
corr_matrix = data.corr()
corr_matrix["rent"].sort_values(ascending=False)

rent             1.000000
bathroom         0.637659
deposit          0.604995
property_size    0.592337
bedroomtotal     0.578228
balconies        0.467774
lift             0.462635
total_floor      0.435476
swimming_pool    0.429813
gym              0.429514
floor            0.309606
longitude        0.161173
lease_type       0.078877
negotiable       0.061501
latitude         0.033619
cup_board       -0.000705
property_age    -0.001435
water_supply    -0.016441
furnishing      -0.102174
parking         -0.493729
Name: rent, dtype: float64

In [85]:
df=data[['bathroom','deposit','property_size','bedroomtotal','balconies','lift','total_floor',
    'swimming_pool','gym',"parking",'rent']]   


In [86]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
X=df.values[:,0:10]
Y=df.values[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [87]:
'''from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
print pd.DataFrame(X_train)'''

'from sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\n# Fit on training set only.\nscaler.fit(X_train)\n# Apply transform to both the training set and the test set.\nX_train = scaler.transform(X_train)\nX_test = scaler.transform(X_test)\nfrom sklearn.decomposition import PCA\n# Make an instance of the Model\npca = PCA(.95)\npca.fit(X_train)\nX_train = pca.transform(X_train)\nX_test = pca.transform(X_test)\nprint pd.DataFrame(X_train)'

In [88]:
def randomforest(input1):

    from sklearn.ensemble import RandomForestRegressor
    forest_reg = RandomForestRegressor()
    forest_reg.fit(X_train, y_train)
    #print('Random Forest R squared score": %.4f' % forest_reg.score((X_test), y_test))
    y_pred = forest_reg.predict(input1)
    print(y_pred)
    #error=abs(y_pred-y_test[])/y_test[]
    #print(error)
    #forest_mse = mean_squared_error(y_pred, y_test)
    #forest_rmse = np.sqrt(forest_mse)
    #print('Random Forest RMSE: %.4f' % forest_rmse)
    #print(y_test[471])
    return y_pred

In [89]:
#randomforest(X_test)